# Basic PID self-driving    

## Helper functions

In [2]:
import carla
import time
import math

def validate_figure8_waypoints(world):
    """Get and validate waypoints for figure-8 track."""
    road_sequence = [6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
    target_lane_id = -4
    waypoint_spacing = 2.0
    
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(waypoint_spacing)
    
    # Filter and sort waypoints
    road_waypoints = {}
    for road_id in road_sequence:
        road_waypoints[road_id] = [wp for wp in waypoints 
                                 if wp.road_id == road_id and wp.lane_id == target_lane_id]
        road_waypoints[road_id].sort(key=lambda x: x.s)
        
        print(f"Road {road_id}: {len(road_waypoints[road_id])} waypoints")
        
        # Validate waypoint spacing
        if len(road_waypoints[road_id]) > 1:
            first = road_waypoints[road_id][0].transform.location
            last = road_waypoints[road_id][-1].transform.location
            distance = ((last.x - first.x)**2 + 
                       (last.y - first.y)**2 + 
                       (last.z - first.z)**2)**0.5
            print(f"Road length: {distance:.1f}m")
    
    return road_waypoints

def drive_figure_eight(world, vehicle):
    """Drive vehicle around Town04's figure-eight."""
    road_waypoints = validate_figure8_waypoints(world)
    road_sequence = [6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
    target_speed = 30  # km/h
    
    try:
        while True:
            for road_id in road_sequence:
                for wp in road_waypoints[road_id]:
                    control = compute_control(vehicle, wp, target_speed)
                    vehicle.apply_control(control)
                    
                    if world.get_settings().synchronous_mode:
                        world.tick()
                    else:
                        time.sleep(0.1)  # Adjust timing as needed
                        
    except KeyboardInterrupt:
        control = carla.VehicleControl(throttle=0, brake=1)
        vehicle.apply_control(control)

def compute_control(vehicle, target_wp, target_speed):
    """Compute vehicle control to reach waypoint."""
    control = carla.VehicleControl()
    
    # Get current state
    current_transform = vehicle.get_transform()
    current_velocity = vehicle.get_velocity()
    speed = 3.6 * math.sqrt(current_velocity.x**2 + current_velocity.y**2)  # km/h
    
    # Steering based on angle difference
    forward = current_transform.get_forward_vector()
    right = current_transform.get_right_vector()
    up = current_transform.get_up_vector()
    target_vector = target_wp.transform.location - current_transform.location
    
    forward_dot = (forward.x * target_vector.x + forward.y * target_vector.y) 
    right_dot = (right.x * target_vector.x + right.y * target_vector.y)
    
    steering = math.atan2(right_dot, forward_dot) / math.pi
    control.steer = max(-1.0, min(1.0, steering))
    
    # Speed control
    if speed < target_speed:
        control.throttle = 0.7
        control.brake = 0.0
    else:
        control.throttle = 0.0
        control.brake = 0.3
        
    return control

def setup_test_environment(world):
    # Enable synchronous mode
    settings = world.get_settings()
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 0.05
    world.apply_settings(settings)
    
    # Find suitable spawn point near road_id 6 (start of figure-8)
    carla_map = world.get_map()
    waypoint = carla_map.get_waypoint(
        carla_map.get_spawn_points()[0].location,
        project_to_road=True,
        lane_type=carla.LaneType.Driving
    )
    
    # Navigate to road_id 6, lane_id -4
    while waypoint.road_id != 6 or waypoint.lane_id != -4:
        next_waypoints = waypoint.next(2.0)
        if not next_waypoints:
            raise RuntimeError("Could not find path to figure-8 road")
        waypoint = next_waypoints[0]
    
    # Spawn vehicle
    blueprint = world.get_blueprint_library().filter('vehicle.*')[0]
    transform = waypoint.transform
    transform.location.z += 0.5  # Prevent spawn collision
    vehicle = world.spawn_actor(blueprint, transform)
    
    return vehicle

## start self driving

In [3]:
# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 
world = client.get_world() 

# load Town04 map
world = client.load_world('Town04')

# Initialize and validate waypoints
road_waypoints = validate_figure8_waypoints(world)

# Setup vehicle in correct position
vehicle = setup_test_environment(world)

# Start driving
drive_figure_eight(world, vehicle)

Road 6: 32 waypoints
Road length: 62.1m
Road 35: 143 waypoints
Road length: 225.2m
Road 36: 41 waypoints
Road length: 80.0m
Road 37: 21 waypoints
Road length: 36.6m
Road 38: 123 waypoints
Road length: 233.5m
Road 39: 67 waypoints
Road length: 132.0m
Road 40: 119 waypoints
Road length: 236.3m
Road 41: 105 waypoints
Road length: 198.2m
Road 42: 17 waypoints
Road length: 32.0m
Road 43: 17 waypoints
Road length: 32.2m
Road 45: 293 waypoints
Road length: 499.0m
Road 46: 70 waypoints
Road length: 142.1m
Road 47: 57 waypoints
Road length: 112.0m
Road 48: 32 waypoints
Road length: 62.0m
Road 49: 20 waypoints
Road length: 37.8m
Road 50: 89 waypoints
Road length: 150.8m
